In [1]:
using KerbalMath
using KerbalRemoteLogging
using KRPC
using SpaceLib
using .Engine
using .Parachute
using .Decoupler
import KRPC.Interface.SpaceCenter as SC
import KRPC.Interface.SpaceCenter.RemoteTypes as SCR
import KRPC.Interface.SpaceCenter.Helpers as SCH


In [2]:
include("setup.jl")
;

In [3]:
sc = SpaceCenter("Karman 3", "10.0.0.51")
sp = add_active_vessel!(sc)
setup_logger(sp)
con = subcontrol(sp, "subcontrol")
core, e1, e2, chute, dec1, dec2 = setup(sp, con)
;

┌ Info: Hardware control loop starting
└ @ SpaceLib /home/rhahi/.julia/dev/SpaceLib/src/spacecraft.jl:217
┌ Info: Acquired spacecraft Karman 3 for control
└ @ SpaceLib /home/rhahi/.julia/dev/SpaceLib/src/spacecenter.jl:56


In [4]:

function stage0(sp::Spacecraft)
    stage!(sp)
    delay(sp.ts, 0.75, "SRB")
    stage!(sp)
    delay(sp.ts, 0.6, "SRB")
end

function stage1(sp::Spacecraft)
    ignite!(sp, e1; name="E1 ignition")
    task = @async wait_for_burnout(sp, e1;
        progress=true,
        name="Stage 1",
        margin=0.33,
        timeout=60,
    )
    stage!(sp)
    wait(task)
end

function stage2(sp::Spacecraft)
    ignite!(sp, e2; name="E2 ignition")
    task = @async wait_for_burnout(sp, e2;
        progress=true,
        name="Stage 2",
        margin=0.5,
        timeout=60,
    )
    shutdown!(e1)
    stage!(sp)
    wait(task)
end

function deploy(sp::Spacecraft)
    periodic_subscribe(sp.ts, 5) do clock
        ref = ReferenceFrame.BCBF(sp.ves)
        h0 = SCH.Position(sp.ves, ref) |> norm
        h_prev = h0
        for now = clock
            h = SCH.Position(sp.ves, ref) |> norm
            h - h_prev < 0 && break
            h_prev = h
            @info "relative height: $(h-h0)"
            yield()
        end
    end
    arm!(chute)
    delay(sp.ts, 1)
    if isarmed(chute)
        @info "Parachute has been armed" _group=:module
    end
end

function detach(sp::Spacecraft)
    decouple!(dec1; top=true)
    decouple!(dec2; bottom=true)
end

detach (generic function with 1 method)

In [5]:
stage0(sp)
stage1(sp)
stage2(sp)
deploy(sp)
detach(sp)